# Yelp API - Lab

The previously deployed lab on working around building a GIS with Yelp API and Folium can be found [here](https://github.com/learn-co-curriculum/dsc-2-15-10-yelp-api-gis-lab/tree/a56358c2d0c2daf569a5f50937c4c27463aadb1a) (not relevant for new students).


## Introduction 

Now that we've seen how the Yelp API works, and some basic Folium visualizations its time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system on to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you, but make the request below.

In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
client_id ='GbRlprkRRJC7wZGTlPFPhQ' #Your client ID goes here (as a string)
api_key = 'RfJ62qbFZ6_GrbBxPwlkQUibYUm4g8bme8ynFhTAdz88pjNOGTsaM3LQWWqD8T2iHrSqb4BLZhQ8ClHcHA9XsXfKkc6K-n8YTaIxHEthi_dxCFD0vr4hB7WATpGSXHYx'


In [3]:
#Your code here
term = 'Indian'
location = 'New York'
SEARCH_LIMIT = 50
offset=0

url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT,
                'offset': offset
            }
response = requests.get(url, headers=headers, params=url_params)
print(response)
print(type(response.text))
print(response.text[:1000])

<Response [200]>
<class 'str'>
{"businesses": [{"id": "sCC7-hSdCkNPExejZT9BAQ", "alias": "the-masalawala-new-york-2", "name": "The MasalaWala", "image_url": "https://s3-media4.fl.yelpcdn.com/bphoto/4fxSLoYtYtP-lDIRQZ-LJw/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/the-masalawala-new-york-2?adjust_creative=GbRlprkRRJC7wZGTlPFPhQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=GbRlprkRRJC7wZGTlPFPhQ", "review_count": 1322, "categories": [{"alias": "indpak", "title": "Indian"}, {"alias": "pakistani", "title": "Pakistani"}], "rating": 4.5, "coordinates": {"latitude": 40.72191, "longitude": -73.98666}, "transactions": ["pickup", "restaurant_reservation", "delivery"], "price": "$$", "location": {"address1": "179 Essex St", "address2": "", "address3": "", "city": "New York", "zip_code": "10002", "country": "US", "state": "NY", "display_address": ["179 Essex St", "New York, NY 10002"]}, "phone": "+12123589300", "display_phone": "(212) 358-9300", "distance

## Pagination

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.josn().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [6]:
# Your code here; use a function or loop to retrieve all the results from your original request

In [7]:
def make_yelp_df(url,headers, url_params, num_pages):
    response= requests.get(url, headers=headers, params=url_params)
    df=pd.DataFrame(response.json()['businesses'])
    for i in range(1,num_pages):
        url_params['offset']+=50
        response= requests.get(url, headers=headers, params=url_params)
        df1=pd.DataFrame(response.json()['businesses'])
        df=pd.concat([df,df1])
    return df
        

In [8]:
response= requests.get(url, headers=headers, params=url_params)

In [9]:
df=pd.DataFrame(response.json()['businesses'])

In [10]:
df=make_yelp_df(url, headers, url_params, 20)

In [11]:
len(df)

1000

In [12]:
df['numprice']=df['price'].map(lambda x: len(str(x)))

## Exploratory Analysis

Take the restaurants from the previous question and do an intial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [56]:
dfh=df[df.rating>4]

In [24]:
def corr(a,b, wcol=1):
    am= a.mean()
    bm=b.mean()
    
    Sxx= sum(wcol*wcol* (a-am)**2)
    Syy= sum(wcol*wcol * (b-bm)**2)
    Sxy= sum(wcol*wcol * (a-am)*(b-bm))
        
    return Sxy/(Sxx*Syy)**.5

In [25]:
list(df.columns)

['alias',
 'categories',
 'coordinates',
 'display_phone',
 'distance',
 'id',
 'image_url',
 'is_closed',
 'location',
 'name',
 'phone',
 'price',
 'rating',
 'review_count',
 'transactions',
 'url',
 'numprice']

In [57]:
corrs=[]
bh=dfh
cols=['rating', 'review_count', 'numprice']

for i in range (len(cols)):
    for j in range(i):
           corrs.append([cols[i], cols[j], wcorr(bh[cols[i]], bh[cols[j]], np.array(bh['review_count'])**1)]) 




In [58]:
corrs

[['review_count', 'rating', -0.7768167924839339],
 ['numprice', 'rating', 0.3734260140680298],
 ['numprice', 'review_count', -0.330287713885725]]

In [121]:
df.head()

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url,numprice
0,the-masalawala-new-york-2,"[{'alias': 'indpak', 'title': 'Indian'}, {'ali...","{'latitude': 40.72191, 'longitude': -73.98666}",(212) 358-9300,1941.690758,sCC7-hSdCkNPExejZT9BAQ,https://s3-media4.fl.yelpcdn.com/bphoto/4fxSLo...,False,"{'address1': '179 Essex St', 'address2': '', '...",The MasalaWala,+12123589300,$$,4.5,1322,"[pickup, delivery, restaurant_reservation]",https://www.yelp.com/biz/the-masalawala-new-yo...,2
1,bengal-tiger-indian-food-new-york,"[{'alias': 'indpak', 'title': 'Indian'}]","{'latitude': 40.763167, 'longitude': -73.977131}",(212) 265-2703,6579.278643,x0k5kFArHLijdY8-NEQ7Xg,https://s3-media1.fl.yelpcdn.com/bphoto/skrQkD...,False,"{'address1': '58 W 56th St', 'address2': 'Fl 2...",Bengal Tiger Indian Food,+12122652703,$$,4.5,1470,"[pickup, delivery]",https://www.yelp.com/biz/bengal-tiger-indian-f...,2
2,tikka-indian-grill-brooklyn-4,"[{'alias': 'indpak', 'title': 'Indian'}, {'ali...","{'latitude': 40.71441, 'longitude': -73.96078}",(718) 768-2262,2998.452269,WWpOWqOgS9ClYvxYVzR0lg,https://s3-media3.fl.yelpcdn.com/bphoto/jZdiyq...,False,"{'address1': '185 Grand St', 'address2': '', '...",Tikka Indian Grill,+17187682262,$$,4.5,274,"[pickup, delivery]",https://www.yelp.com/biz/tikka-indian-grill-br...,2
3,seva-indian-cuisine-astoria-5,"[{'alias': 'indpak', 'title': 'Indian'}]","{'latitude': 40.7653633, 'longitude': -73.9191...",(718) 626-4440,9206.478614,lxNU-aBq0yPwBDx-i89ahA,https://s3-media4.fl.yelpcdn.com/bphoto/MsQyig...,False,"{'address1': '3007 34th St', 'address2': '', '...",Seva Indian Cuisine,+17186264440,$$,4.5,1114,"[pickup, delivery]",https://www.yelp.com/biz/seva-indian-cuisine-a...,2
4,mughlai-indian-cuisine-new-york-4,"[{'alias': 'indpak', 'title': 'Indian'}]","{'latitude': 40.78002, 'longitude': -73.94978}",(212) 722-6262,9100.989031,izRJeBfDQYJmd6ibIQ6Mfg,https://s3-media2.fl.yelpcdn.com/bphoto/DNsKer...,False,"{'address1': '1724 2nd Ave', 'address2': '', '...",Mughlai Indian Cuisine,+12127226262,$$,4.5,385,"[pickup, delivery]",https://www.yelp.com/biz/mughlai-indian-cuisin...,2


In [126]:
import folium

lat = 40.7
long = -74

#Create a map of the area
base_map = folium.Map([lat, long], zoom_start=13)
base_map

In [ ]:
for p in df.coordinates:
    lat = p['latitude']
    long = p['longitude']
    popup_text = "Lattitude: {}, Longitude: {}".format(lat,long)
    popup = folium.Popup(popup_text, parse_html=True)
    marker = folium.Marker(location=[lat, long], popup=popup)
    marker.add_to(base_map)
base_map

## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [ ]:
#Your code here

## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!